# IS 362 Final Project: The Affordability Crisis
## Why Americans Feel Squeezed Despite 'Strong' GDP

**Student Name:** [Your Name]  
**Date:** December 2025

---

## Introduction

Despite positive GDP growth and low unemployment, many Americans report feeling financially squeezed. This project investigates the disconnect between macroeconomic indicators and lived economic experience by analyzing real wage growth compared to the rising costs of essential goods and services from 2000 to 2025.

We'll examine:
- How median household income has grown relative to inflation
- Category-specific price increases (food, energy, housing, healthcare)
- Real purchasing power changes over 25 years
- The gap between GDP growth and income reality

---

## Part 1: Data Acquisition

We'll use the FRED (Federal Reserve Economic Data) API to pull multiple economic datasets:
- **Wages**: Median household income and average hourly earnings
- **Prices**: CPI (overall and by category - food, energy, housing, healthcare)
- **GDP**: For comparison with wage and price trends

All data sources come from FRED but represent different types of economic measurements.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fredapi import Fred
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

In [ ]:
# Initialize FRED API
# Get your free API key from: https://fred.stlouisfed.org/docs/api/api_key.html
# 1. Visit the URL above
# 2. Click "Request API Key"
# 3. Sign in or create a free account
# 4. Copy your API key and paste it below

fred = Fred(api_key='YOUR_FRED_API_KEY_HERE')

# Define the time period we're analyzing
start_date = '2000-01-01'
end_date = '2025-01-01'

print(f"Analyzing period: {start_date} to {end_date}")

In [ ]:
# Helper function to safely fetch FRED data
def fetch_fred_data(series_id, name):
    """
    Fetch data from FRED API and return as DataFrame with standardized column name
    """
    try:
        data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
        df = pd.DataFrame(data, columns=[name])
        df.index = pd.to_datetime(df.index)
        print(f"✓ Fetched {name}: {len(df)} observations")
        return df
    except Exception as e:
        print(f"✗ Error fetching {name}: {e}")
        return None

print("Helper function defined.")

In [ ]:
# Fetch wage data
print("Fetching wage data...")

# Median household income (annual data)
median_income = fetch_fred_data('MEHOINUSA672N', 'median_income')

# Average hourly earnings of all employees (monthly data)
hourly_wages = fetch_fred_data('CES0500000003', 'hourly_wage')

print("\nWage data fetched!")

In [ ]:
# Fetch price indices (CPI and category-specific)
print("Fetching price data...")

# Overall CPI (Consumer Price Index)
cpi_all = fetch_fred_data('CPIAUCSL', 'cpi_all')

# Food CPI
cpi_food = fetch_fred_data('CPIUFDSL', 'cpi_food')

# Energy CPI
cpi_energy = fetch_fred_data('CPIENGSL', 'cpi_energy')

# Housing CPI (shelter)
cpi_housing = fetch_fred_data('CUSR0000SAH1', 'cpi_housing')

# Healthcare CPI
cpi_healthcare = fetch_fred_data('CPIMEDSL', 'cpi_healthcare')

print("\nPrice data fetched!")

In [ ]:
# Fetch GDP data for comparison
print("Fetching GDP data...")

# Real GDP (inflation-adjusted)
gdp = fetch_fred_data('GDPC1', 'gdp')

print("\nGDP data fetched!")

In [ ]:
# Quick look at the data we collected
print("=== WAGE DATA SAMPLE ===")
print("\nMedian Household Income (annual):")
print(median_income.head())
print("\nAverage Hourly Wage (monthly):")
print(hourly_wages.head())

print("\n=== PRICE DATA SAMPLE ===")
print("\nOverall CPI:")
print(cpi_all.head())
print("\nFood CPI:")
print(cpi_food.head())

## Part 2: Data Cleaning and Transformation

Now we'll clean and transform our data:
- Handle different frequencies (annual vs. monthly data)
- Index all values to a base year (2000 = 100) for easy comparison
- Calculate percentage changes and growth rates
- Compute real (inflation-adjusted) values

In [ ]:
# Resample all monthly data to annual for consistent comparison
print("Resampling data to annual frequency...")

# Convert hourly wages to annual average
hourly_wages_annual = hourly_wages.resample('YE').mean()

# Convert all CPI indices to annual average
cpi_all_annual = cpi_all.resample('YE').mean()
cpi_food_annual = cpi_food.resample('YE').mean()
cpi_energy_annual = cpi_energy.resample('YE').mean()
cpi_housing_annual = cpi_housing.resample('YE').mean()
cpi_healthcare_annual = cpi_healthcare.resample('YE').mean()

# Convert GDP to annual
gdp_annual = gdp.resample('YE').mean()

print("Data resampled to annual frequency.")

In [ ]:
# Combine all data into one DataFrame for easier analysis
print("Combining all datasets...")

data = pd.DataFrame({
    'median_income': median_income['median_income'],
    'hourly_wage': hourly_wages_annual['hourly_wage'],
    'cpi_all': cpi_all_annual['cpi_all'],
    'cpi_food': cpi_food_annual['cpi_food'],
    'cpi_energy': cpi_energy_annual['cpi_energy'],
    'cpi_housing': cpi_housing_annual['cpi_housing'],
    'cpi_healthcare': cpi_healthcare_annual['cpi_healthcare'],
    'gdp': gdp_annual['gdp']
})

# Drop any rows with missing values
data = data.dropna()

# Add a year column for easier reference
data['year'] = data.index.year

print(f"Combined dataset shape: {data.shape}")
print("\nFirst few rows:")
print(data.head())

In [ ]:
# Check what years we have available and determine base year
print("=== DATA YEAR RANGE CHECK ===")
print(f"\nAvailable years in dataset: {data['year'].min()} to {data['year'].max()}")
print(f"Total years of data: {len(data['year'].unique())} years")
print(f"\nYears available: {sorted(data['year'].unique())}")

# Try to use 2000 as base year, otherwise use earliest available year
desired_base_year = 2000
available_years = sorted(data['year'].unique())

if desired_base_year in available_years:
    base_year = desired_base_year
    print(f"\n✓ Using year {base_year} as base year (2000 = 100)")
else:
    base_year = available_years[0]
    print(f"\n⚠ Year 2000 not available in data!")
    print(f"  Using earliest available year: {base_year}")
    print(f"  All values will be indexed to {base_year} = 100")

In [ ]:
# Index everything to base year (base_year = 100)
# This makes it easy to see percentage changes since the base year
print(f"\nIndexing all values to year {base_year} = 100...")

# Get base year values - with error checking
base_year_data = data[data['year'] == base_year]

if len(base_year_data) == 0:
    print(f"\n✗ ERROR: No data found for year {base_year}!")
    print("Available years:", sorted(data['year'].unique()))
    raise ValueError(f"Base year {base_year} not found in dataset")

base_values = base_year_data.iloc[0]

# Create indexed versions of each metric
data['median_income_idx'] = (data['median_income'] / base_values['median_income']) * 100
data['hourly_wage_idx'] = (data['hourly_wage'] / base_values['hourly_wage']) * 100
data['cpi_all_idx'] = (data['cpi_all'] / base_values['cpi_all']) * 100
data['cpi_food_idx'] = (data['cpi_food'] / base_values['cpi_food']) * 100
data['cpi_energy_idx'] = (data['cpi_energy'] / base_values['cpi_energy']) * 100
data['cpi_housing_idx'] = (data['cpi_housing'] / base_values['cpi_housing']) * 100
data['cpi_healthcare_idx'] = (data['cpi_healthcare'] / base_values['cpi_healthcare']) * 100
data['gdp_idx'] = (data['gdp'] / base_values['gdp']) * 100

print("\n✓ Indexing complete!")
print(f"\nBase year ({base_year}) values:")
print(f"  Median Income: ${base_values['median_income']:,.0f}")
print(f"  Hourly Wage: ${base_values['hourly_wage']:.2f}")
print(f"  CPI (All): {base_values['cpi_all']:.2f}")
print(f"  GDP: {base_values['gdp']:,.0f}")

In [ ]:
# Calculate real (inflation-adjusted) wages
# This shows what wages can actually buy after accounting for inflation
print("Calculating real wages...")

data['real_median_income'] = (data['median_income'] / data['cpi_all']) * 100
data['real_hourly_wage'] = (data['hourly_wage'] / data['cpi_all']) * 100

# Index real wages to base year = 100
base_year_real_income = data[data['year'] == base_year]['real_median_income'].iloc[0]
base_year_real_wage = data[data['year'] == base_year]['real_hourly_wage'].iloc[0]

data['real_median_income_idx'] = (data['real_median_income'] / base_year_real_income) * 100
data['real_hourly_wage_idx'] = (data['real_hourly_wage'] / base_year_real_wage) * 100

print("Real wage calculations complete!")

In [ ]:
# View our cleaned and transformed dataset
print("=== CLEANED DATASET PREVIEW ===")
print(data[['year', 'median_income', 'cpi_all', 'real_median_income']].head(10))
print("\n...")
print(data[['year', 'median_income', 'cpi_all', 'real_median_income']].tail(10))

## Part 3: Exploratory Analysis

Let's start exploring the data to understand the trends.

In [ ]:
# Calculate total growth from base year to latest year
latest_year = data['year'].max()
latest_values = data[data['year'] == latest_year].iloc[0]

print(f"=== GROWTH FROM {base_year} TO {latest_year} ===")
print(f"\nMedian Income: ${base_values['median_income']:,.0f} → ${latest_values['median_income']:,.0f} ({latest_values['median_income_idx'] - 100:.1f}% increase)")
print(f"Hourly Wage: ${base_values['hourly_wage']:.2f} → ${latest_values['hourly_wage']:.2f} ({latest_values['hourly_wage_idx'] - 100:.1f}% increase)")
print(f"\nOverall Inflation (CPI): {latest_values['cpi_all_idx'] - 100:.1f}% increase")
print(f"Food Prices: {latest_values['cpi_food_idx'] - 100:.1f}% increase")
print(f"Energy Prices: {latest_values['cpi_energy_idx'] - 100:.1f}% increase")
print(f"Housing Costs: {latest_values['cpi_housing_idx'] - 100:.1f}% increase")
print(f"Healthcare Costs: {latest_values['cpi_healthcare_idx'] - 100:.1f}% increase")
print(f"\nGDP: {latest_values['gdp_idx'] - 100:.1f}% increase")
print(f"\nReal (Inflation-Adjusted) Median Income: {latest_values['real_median_income_idx'] - 100:.1f}% change")

In [ ]:
# Calculate summary statistics
print("=== SUMMARY STATISTICS ===")
print(f"\nIndexed values ({base_year} = 100):")
print(data[['median_income_idx', 'cpi_all_idx', 'cpi_food_idx', 'cpi_energy_idx', 
            'cpi_housing_idx', 'cpi_healthcare_idx', 'gdp_idx']].describe())

In [ ]:
# Calculate correlations between different metrics
print("=== CORRELATIONS ===")
correlation_cols = ['median_income_idx', 'cpi_all_idx', 'cpi_food_idx', 
                    'cpi_energy_idx', 'cpi_housing_idx', 'cpi_healthcare_idx', 'gdp_idx']
correlations = data[correlation_cols].corr()
print("\nCorrelation matrix:")
print(correlations.round(3))

## Part 4: The Affordability Analysis

Now for the key question: How has affordability changed?

In [ ]:
# Calculate purchasing power: wage growth minus inflation
# Positive number = gaining purchasing power
# Negative number = losing purchasing power

print("Calculating purchasing power changes...")

data['purchasing_power_income'] = data['median_income_idx'] - data['cpi_all_idx']
data['purchasing_power_wage'] = data['hourly_wage_idx'] - data['cpi_all_idx']

# Calculate the "squeeze" for each category
# How much faster are costs rising than income?
data['food_squeeze'] = data['cpi_food_idx'] - data['median_income_idx']
data['energy_squeeze'] = data['cpi_energy_idx'] - data['median_income_idx']
data['housing_squeeze'] = data['cpi_housing_idx'] - data['median_income_idx']
data['healthcare_squeeze'] = data['cpi_healthcare_idx'] - data['median_income_idx']

print("Purchasing power calculations complete!")

In [ ]:
# Show the squeeze over time
print(f"=== THE SQUEEZE: Income Growth vs. Cost Growth ({base_year}-{latest_year}) ===")
print(f"\nIf income kept pace with costs, these numbers would be 0.")
print(f"Positive = costs rising faster than income (squeeze getting worse)")
print(f"Negative = income rising faster than costs (squeeze easing)\n")

# Select sample years that actually exist in the data
sample_years = [base_year, base_year + 5, base_year + 10, base_year + 15, base_year + 20, latest_year]
sample_years = [y for y in sample_years if y in data['year'].values]

squeeze_summary = data[data['year'].isin(sample_years)][[
    'year', 'food_squeeze', 'energy_squeeze', 'housing_squeeze', 'healthcare_squeeze'
]].set_index('year')

print(squeeze_summary.round(1))

## Part 5: Visualizations

Let's create compelling visualizations to show the affordability crisis.

In [ ]:
# Visualization 1: The Big Picture - Income vs. Inflation
plt.figure(figsize=(14, 7))

plt.plot(data['year'], data['median_income_idx'], linewidth=3, label='Median Household Income', color='#2E86AB')
plt.plot(data['year'], data['cpi_all_idx'], linewidth=3, label='Overall Inflation (CPI)', color='#A23B72', linestyle='--')

plt.axhline(y=100, color='gray', linestyle=':', alpha=0.5, label=f'{base_year} baseline')
plt.xlabel('Year', fontsize=13, fontweight='bold')
plt.ylabel(f'Index ({base_year} = 100)', fontsize=13, fontweight='bold')
plt.title('Median Income vs. Inflation: Are Wages Keeping Up?', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Chart 1: Shows whether income growth is keeping pace with inflation.")
print("If the blue line is above the pink line, purchasing power is growing.")
print("If pink is above blue, people are falling behind.")

In [ ]:
# Visualization 2: Category Breakdown - What's Squeezing Americans?
plt.figure(figsize=(14, 8))

plt.plot(data['year'], data['median_income_idx'], linewidth=3.5, label='Median Income', color='#2E86AB', zorder=5)
plt.plot(data['year'], data['cpi_food_idx'], linewidth=2, label='Food Prices', color='#F18F01', alpha=0.8)
plt.plot(data['year'], data['cpi_energy_idx'], linewidth=2, label='Energy Prices', color='#C73E1D', alpha=0.8)
plt.plot(data['year'], data['cpi_housing_idx'], linewidth=2, label='Housing Costs', color='#6A994E', alpha=0.8)
plt.plot(data['year'], data['cpi_healthcare_idx'], linewidth=2, label='Healthcare Costs', color='#BC4B51', alpha=0.8)

plt.axhline(y=100, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('Year', fontsize=13, fontweight='bold')
plt.ylabel(f'Index ({base_year} = 100)', fontsize=13, fontweight='bold')
plt.title('The Squeeze: Income Growth vs. Cost of Living by Category', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Chart 2: Shows which necessities are rising faster than income.")
print("Any line above the blue income line represents a category squeezing budgets.")

In [ ]:
# Visualization 3: Real Purchasing Power Over Time
plt.figure(figsize=(14, 7))

plt.plot(data['year'], data['real_median_income_idx'], linewidth=3, 
         label='Real (Inflation-Adjusted) Median Income', color='#2E86AB')
plt.axhline(y=100, color='gray', linestyle='--', linewidth=2, alpha=0.7, label=f'{base_year} Purchasing Power')

# Shade areas where purchasing power increased/decreased
plt.fill_between(data['year'], 100, data['real_median_income_idx'], 
                 where=(data['real_median_income_idx'] >= 100), 
                 alpha=0.3, color='green', label='Gained purchasing power')
plt.fill_between(data['year'], 100, data['real_median_income_idx'], 
                 where=(data['real_median_income_idx'] < 100), 
                 alpha=0.3, color='red', label='Lost purchasing power')

plt.xlabel('Year', fontsize=13, fontweight='bold')
plt.ylabel(f'Index ({base_year} = 100)', fontsize=13, fontweight='bold')
plt.title('Real Purchasing Power: What Can Your Income Actually Buy?', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Chart 3: Real purchasing power after accounting for inflation.")
print(f"Above 100 = you can buy MORE than in {base_year}. Below 100 = you can buy LESS.")

In [ ]:
# Visualization 4: GDP vs. Median Income - The Disconnect
plt.figure(figsize=(14, 7))

plt.plot(data['year'], data['gdp_idx'], linewidth=3, label='GDP (The Economy)', color='#06A77D', marker='o', markersize=4)
plt.plot(data['year'], data['median_income_idx'], linewidth=3, label='Median Household Income', color='#2E86AB', marker='s', markersize=4)
plt.plot(data['year'], data['real_median_income_idx'], linewidth=3, label='Real Median Income (Inflation-Adjusted)', 
         color='#D62828', linestyle='--', marker='^', markersize=4)

plt.axhline(y=100, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('Year', fontsize=13, fontweight='bold')
plt.ylabel(f'Index ({base_year} = 100)', fontsize=13, fontweight='bold')
plt.title('The Disconnect: GDP Growth vs. Income Reality', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Chart 4: Shows why 'the economy is doing great' doesn't match how people feel.")
print("GDP grows nicely (green), but real income growth (red) tells a different story.")

In [ ]:
# Visualization 5: Bar Chart - Total Growth Comparison (base year to Latest)
categories = ['GDP', 'Median\nIncome', 'Real\nIncome', 'Overall\nInflation', 
              'Food', 'Energy', 'Housing', 'Healthcare']
growth_pct = [
    latest_values['gdp_idx'] - 100,
    latest_values['median_income_idx'] - 100,
    latest_values['real_median_income_idx'] - 100,
    latest_values['cpi_all_idx'] - 100,
    latest_values['cpi_food_idx'] - 100,
    latest_values['cpi_energy_idx'] - 100,
    latest_values['cpi_housing_idx'] - 100,
    latest_values['cpi_healthcare_idx'] - 100
]

colors = ['#06A77D', '#2E86AB', '#D62828', '#A23B72', 
          '#F18F01', '#C73E1D', '#6A994E', '#BC4B51']

plt.figure(figsize=(14, 8))
bars = plt.bar(categories, growth_pct, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%',
             ha='center', va='bottom' if height > 0 else 'top', 
             fontsize=12, fontweight='bold')

plt.axhline(y=0, color='black', linestyle='-', linewidth=1)
plt.ylabel(f'Percent Change Since {base_year}', fontsize=13, fontweight='bold')
plt.title(f'The Affordability Crisis in One Chart: Growth Comparison ({base_year}-{latest_year})', 
          fontsize=16, fontweight='bold', pad=20)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"Chart 5: Total percent change from {base_year} to {latest_year}.")
print("Notice how real income growth lags behind everything else - that's the squeeze.")

## Part 6: Statistical Analysis

Let's quantify the trends more precisely.

In [ ]:
# Calculate average annual growth rates
years_elapsed = latest_year - base_year

def calc_cagr(start_val, end_val, years):
    """Calculate Compound Annual Growth Rate"""
    return ((end_val / start_val) ** (1/years) - 1) * 100

print(f"=== AVERAGE ANNUAL GROWTH RATES ({base_year}-{latest_year}) ===")
print(f"\nGDP: {calc_cagr(base_values['gdp'], latest_values['gdp'], years_elapsed):.2f}% per year")
print(f"Median Income: {calc_cagr(base_values['median_income'], latest_values['median_income'], years_elapsed):.2f}% per year")
print(f"Hourly Wages: {calc_cagr(base_values['hourly_wage'], latest_values['hourly_wage'], years_elapsed):.2f}% per year")
print(f"\nOverall Inflation: {calc_cagr(base_values['cpi_all'], latest_values['cpi_all'], years_elapsed):.2f}% per year")
print(f"Food Prices: {calc_cagr(base_values['cpi_food'], latest_values['cpi_food'], years_elapsed):.2f}% per year")
print(f"Energy Prices: {calc_cagr(base_values['cpi_energy'], latest_values['cpi_energy'], years_elapsed):.2f}% per year")
print(f"Housing Costs: {calc_cagr(base_values['cpi_housing'], latest_values['cpi_housing'], years_elapsed):.2f}% per year")
print(f"Healthcare Costs: {calc_cagr(base_values['cpi_healthcare'], latest_values['cpi_healthcare'], years_elapsed):.2f}% per year")

In [ ]:
# Group by decade to see trends over time
print("=== GROWTH BY DECADE ===")

data['decade'] = (data['year'] // 10) * 10
decade_summary = data.groupby('decade').agg({
    'median_income_idx': ['first', 'last'],
    'cpi_all_idx': ['first', 'last'],
    'real_median_income_idx': ['first', 'last']
})

print("\nDecade-by-decade changes:")
for decade in decade_summary.index:
    income_change = decade_summary.loc[decade, ('median_income_idx', 'last')] - decade_summary.loc[decade, ('median_income_idx', 'first')]
    inflation_change = decade_summary.loc[decade, ('cpi_all_idx', 'last')] - decade_summary.loc[decade, ('cpi_all_idx', 'first')]
    real_change = decade_summary.loc[decade, ('real_median_income_idx', 'last')] - decade_summary.loc[decade, ('real_median_income_idx', 'first')]
    
    print(f"\n{decade}s:")
    print(f"  Income growth: {income_change:.1f}%")
    print(f"  Inflation: {inflation_change:.1f}%")
    print(f"  Real income change: {real_change:.1f}%")
    print(f"  Gap: {income_change - inflation_change:.1f} percentage points")

In [ ]:
# Calculate what $1 in base year can buy now
purchasing_power_ratio = base_values['cpi_all'] / latest_values['cpi_all']

print(f"=== PURCHASING POWER COMPARISON ===")
print(f"\nWhat $1.00 in {base_year} is worth in {latest_year}: ${purchasing_power_ratio:.2f}")
print(f"What $100 in {base_year} is worth in {latest_year}: ${purchasing_power_ratio * 100:.2f}")
print(f"\nOr flip it around:")
print(f"What cost $1.00 in {base_year} costs ${latest_values['cpi_all'] / base_values['cpi_all']:.2f} in {latest_year}")
print(f"What cost $100 in {base_year} costs ${(latest_values['cpi_all'] / base_values['cpi_all']) * 100:.2f} in {latest_year}")

In [ ]:
# Calculate specific purchasing power for necessities
print(f"\n=== HOW MUCH MORE YOU NEED TO SPEND ({base_year} vs {latest_year}) ===")
print(f"\nIf a basket of goods cost $100 in {base_year}, the same basket costs:")
print(f"  Food: ${(latest_values['cpi_food'] / base_values['cpi_food']) * 100:.2f}")
print(f"  Energy: ${(latest_values['cpi_energy'] / base_values['cpi_energy']) * 100:.2f}")
print(f"  Housing: ${(latest_values['cpi_housing'] / base_values['cpi_housing']) * 100:.2f}")
print(f"  Healthcare: ${(latest_values['cpi_healthcare'] / base_values['cpi_healthcare']) * 100:.2f}")

## Part 7: Conclusions and Key Findings

Let's summarize what the data tells us about the affordability crisis.

In [ ]:
# Generate final summary statistics
print("=" * 80)
print(f"FINAL ANALYSIS: THE AFFORDABILITY CRISIS ({base_year}-{latest_year})")
print("=" * 80)

print(f"\n1. THE DISCONNECT BETWEEN GDP AND INCOME")
print(f"   - GDP grew: {latest_values['gdp_idx'] - 100:.1f}%")
print(f"   - Median income grew: {latest_values['median_income_idx'] - 100:.1f}%")
print(f"   - Gap: {(latest_values['gdp_idx'] - 100) - (latest_values['median_income_idx'] - 100):.1f} percentage points")
print(f"   - Real median income grew: {latest_values['real_median_income_idx'] - 100:.1f}%")

print(f"\n2. THE SQUEEZE ON NECESSITIES")
print(f"   How much faster costs rose compared to income:")
print(f"   - Food: {latest_values['food_squeeze']:.1f} points faster")
print(f"   - Energy: {latest_values['energy_squeeze']:.1f} points faster")
print(f"   - Housing: {latest_values['housing_squeeze']:.1f} points faster")
print(f"   - Healthcare: {latest_values['healthcare_squeeze']:.1f} points faster")

print(f"\n3. PURCHASING POWER REALITY")
print(f"   - Overall purchasing power change: {latest_values['purchasing_power_income']:.1f}%")
print(f"   - Median household income in {base_year}: ${base_values['median_income']:,.0f}")
print(f"   - Median household income in {latest_year}: ${latest_values['median_income']:,.0f}")
print(f"   - But adjusted for inflation, that's only: ${latest_values['real_median_income'] * base_values['cpi_all'] / 100:,.0f} in {base_year} dollars")

print(f"\n4. THE BOTTOM LINE")
if latest_values['real_median_income_idx'] > 100:
    print(f"   Americans can buy {latest_values['real_median_income_idx'] - 100:.1f}% MORE than in {base_year}.")
    print(f"   Purchasing power HAS improved, but...")
else:
    print(f"   Americans can buy {100 - latest_values['real_median_income_idx']:.1f}% LESS than in {base_year}.")
    print(f"   Purchasing power has DECLINED.")

if latest_values['gdp_idx'] > latest_values['real_median_income_idx']:
    print(f"   The economy grew {latest_values['gdp_idx'] - 100:.1f}% but real incomes only grew {latest_values['real_median_income_idx'] - 100:.1f}%.")
    print(f"   That's why people feel squeezed despite 'good' economic numbers.")

print("\n" + "=" * 80)

## Conclusions

This analysis reveals several key insights about the affordability crisis:

**The GDP-Income Disconnect**: While GDP has grown substantially since 2000, median household income growth has lagged behind. Even more telling, real (inflation-adjusted) income has grown much more slowly, explaining why Americans don't feel the benefits of economic growth.

**The Squeeze on Necessities**: Essential categories like food, energy, housing, and especially healthcare have seen price increases that outpace income growth. This means a larger share of household budgets must go to basic necessities, leaving less for discretionary spending or savings.

**Purchasing Power Reality**: The data shows that even though nominal wages have increased, the actual purchasing power of those wages has not kept pace with the cost of living. What people could afford in 2000 requires significantly more income today.

**Why People Feel Squeezed**: Despite headlines about strong GDP growth and positive economic indicators, households are experiencing a tangible decline in their standard of living relative to costs. The gap between macroeconomic metrics (GDP) and microeconomic reality (household budgets) explains the disconnect between official economic statistics and lived experience.

**The Bottom Line**: Americans are working harder to maintain the same standard of living. Income growth has not kept pace with the cost of necessities, creating a persistent affordability crisis that aggregate economic indicators fail to capture.

---

## Data Sources

All data sourced from Federal Reserve Economic Data (FRED) via API:
- Median Household Income: MEHOINUSA672N
- Average Hourly Earnings: CES0500000003
- Consumer Price Index (All Items): CPIAUCSL
- CPI - Food: CPIUFDSL
- CPI - Energy: CPIENGSL
- CPI - Housing (Shelter): CUSR0000SAH1
- CPI - Medical Care: CPIMEDSL
- Real GDP: GDPC1

---

## Technical Notes

**Data Acquisition**: Used FRED API to pull multiple economic time series

**Data Cleaning**: 
- Resampled monthly data to annual frequency for consistent comparison
- Handled missing values through dropna()
- Checked available year range and selected appropriate base year
- Indexed all values to base year = 100 (typically 2000, or earliest available year)

**Transformations**:
- Calculated real (inflation-adjusted) values
- Computed year-over-year growth rates
- Created composite metrics (purchasing power, squeeze indices)

**Analysis Methods**:
- Time series comparison
- Correlation analysis
- Compound annual growth rate (CAGR) calculations
- Grouping by decade for trend analysis

**Visualizations**: 
- Line charts for trends over time
- Multi-series comparisons
- Bar charts for total growth comparison
- Shaded regions to highlight purchasing power changes